# Notebook 1: Extract Z-scored LFP

- Notebook for extracting local field potential traces from Spikegadgets `.rec` files

In [1]:
import sys
import os
import git

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/projects/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
from utilities import helper

## Inputs & Data

Explanation of each input and where it comes from.

In [8]:
EPHYS_SAMPLING_RATE = 20000
LFP_SAMPLING_RATE = 1000
TRIAL_DURATION = 10
FRAME_RATE = 22
ECU_STREAM_ID = "ECU"
TRODES_STREAM_ID = "trodes"
LFP_FREQ_MIN = 0.5
LFP_FREQ_MAX = 300
ELECTRIC_NOISE_FREQ = 60
RECORDING_EXTENTION = "*.rec"

In [9]:
ORIGINAL_TRIAL_COL = "condition"
TRIAL_OUTCOME_COL = "trial_outcome"

In [10]:
# NOTE: Change based on individual project data location

# Spreadsheet of channel mapping
CHANNEL_MAPPING_DF = pd.read_excel("../../data/channel_mapping.xlsx")
# Spreadsheet of tone time
TONE_TIMESTAMP_DF = pd.read_excel("../../proc/rce_tone_timestamps.csv", index_col=0)

In [11]:
CHANNEL_MAPPING_DF.head()

,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,6.1,NaN,15,14,13,31,21.0,15.0,14.0,13.0,16.0
1,1,6.2,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
2,1,6.3,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
3,1,6.4,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
4,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0


In [12]:
TONE_TIMESTAMP_DF.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,condition,competition_closeness,Unnamed: 13
2299.0,3772337.0,1.0,20221122_161341_omission_subject_6_1_and_6_3,20221122_161341_omission_subject_6_1_top_4_base_2,dio_ECU_Din1,0.0,20221122_161341_omission_subject_6_1_and_6_3.1...,0.0,1.0,6_1_top_4_base_2,NaN,NaN,NaN
2300.0,5204112.0,1.0,20221122_161341_omission_subject_6_1_and_6_3,20221122_161341_omission_subject_6_1_top_4_base_2,dio_ECU_Din1,1431775.0,20221122_161341_omission_subject_6_1_and_6_3.1...,1784.0,1.0,6_1_top_4_base_2,NaN,NaN,NaN
2301.0,6804107.0,1.0,20221122_161341_omission_subject_6_1_and_6_3,20221122_161341_omission_subject_6_1_top_4_base_2,dio_ECU_Din1,3031770.0,20221122_161341_omission_subject_6_1_and_6_3.1...,3779.0,1.0,6_1_top_4_base_2,NaN,NaN,NaN
2302.0,8604101.0,1.0,20221122_161341_omission_subject_6_1_and_6_3,20221122_161341_omission_subject_6_1_top_4_base_2,dio_ECU_Din1,4831764.0,20221122_161341_omission_subject_6_1_and_6_3.1...,6021.0,1.0,6_1_top_4_base_2,NaN,NaN,NaN
2303.0,10204096.0,1.0,20221122_161341_omission_subject_6_1_and_6_3,20221122_161341_omission_subject_6_1_top_4_base_2,dio_ECU_Din1,6431759.0,20221122_161341_omission_subject_6_1_and_6_3.1...,8015.0,1.0,6_1_top_4_base_2,NaN,NaN,NaN


In [13]:
# NOTE: Change based on individual project data location
# Where all the recording files are being saved
ALL_SESSION_DIR = glob.glob("/scratch/back_up/reward_competition_extention/data/omission/*/*.rec")

In [14]:
ALL_SESSION_DIR

['/scratch/back_up/reward_competition_extention/data/omission/2023_06_17/20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_17/20230617_115641_standard_comp_to_omission_D1_subj_2-2_and_2-4.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2022_12_02/20221202_134600_omission_and_competition_subject_6_1_and_6_2.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_21/20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_20/20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2022_12_03/20221203_154800_omission_and_competition_subject_6_4_and_6_1.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_18/20230618_100646_standard_comp_to_omission_D2_subj_2-4_and_2-1.rec',
 '/scratch/back

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [15]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"../../proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Functions

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [16]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




# Extracting the LFP

In [53]:
recording_name_to_all_ch_lfp = {}
# Going through all the recording sessions 
for session_dir in ALL_SESSION_DIR:
    # Going through all the recordings in each session
    for recording_path in glob.glob(os.path.join(session_dir, RECORDING_EXTENTION)):
        try:
            recording_basename = os.path.splitext(os.path.basename(recording_path))[0]
            # checking to see if the recording has an ECU component
            # if it doesn't, then the next one be extracted
            current_recording = se.read_spikegadgets(recording_path, stream_id=ECU_STREAM_ID)
            current_recording = se.read_spikegadgets(recording_path, stream_id=TRODES_STREAM_ID)
            print(recording_basename)
            # Preprocessing the LFP
            current_recording = sp.bandpass_filter(current_recording, freq_min=LFP_FREQ_MIN, freq_max=LFP_FREQ_MAX)
            current_recording = sp.notch_filter(current_recording, freq=ELECTRIC_NOISE_FREQ)
            current_recording = sp.resample(current_recording, resample_rate=LFP_SAMPLING_RATE)
            current_recording = sp.zscore(current_recording)
            recording_name_to_all_ch_lfp[recording_basename] = current_recording
        except Exception as error:
            # handle the exception
            print("An exception occurred:", error) # An exception occurred: division by zero
    




An exception occurred: stream_id trodes is not in ['ECU']
An exception occurred: stream_id trodes is not in ['ECU']
An exception occurred: stream_id trodes is not in ['ECU']
An exception occurred: stream_id trodes is not in ['ECU']
An exception occurred: stream_id trodes is not in ['ECU']
An exception occurred: stream_id trodes is not in ['ECU']
An exception occurred: stream_id trodes is not in ['ECU']
An exception occurred: stream_id trodes is not in ['ECU']
An exception occurred: stream_id trodes is not in ['ECU']
An exception occurred: stream_id trodes is not in ['ECU']
An exception occurred: stream_id trodes is not in ['ECU']
An exception occurred: stream_id trodes is not in ['ECU']


In [54]:
current_recording.get_traces(channel_ids=["0"], start_frame=0, end_frame=10).T[0]

array([709, 694, 654, 612, 558, 459, 355, 329, 396, 442], dtype=int16)

In [56]:
TONE_TIMESTAMP_DF[TONE_TIMESTAMP_DF["recording_file"].str.contains("1215")]

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,video_name,all_subjects,current_subject,trial_outcome,lfp_index,baseline_lfp_timestamp_range,trial_lfp_timestamp_range,baseline_ephys_timestamp_range,trial_ephys_timestamp_range,baseline_videoframe_range,trial_videoframe_range
99,2921801,20221215_145401_comp_amd_om_6_1_and_6_3,20221215_145401_comp_amd_om_6_1_top_4_base_3,1872697,20221215_145401_comp_amd_om_6_1_and_6_3.1.vide...,2387,1.0,6_1_top_4_base_3,win_non_comp,20221215_145401_comp_amd_om_6_1_and_6_3.1,"(6.1, 6.3)",6.1,win,93634,"(83634, 93634)","(93634, 103634)","(1672697, 1872697)","(1872697, 2072697)","(2167, 2387)","(2387, 2607)"
100,4521798,20221215_145401_comp_amd_om_6_1_and_6_3,20221215_145401_comp_amd_om_6_1_top_4_base_3,3472694,20221215_145401_comp_amd_om_6_1_and_6_3.1.vide...,4382,1.0,6_1_top_4_base_3,win_non_comp,20221215_145401_comp_amd_om_6_1_and_6_3.1,"(6.1, 6.3)",6.1,win,173634,"(163634, 173634)","(173634, 183634)","(3272694, 3472694)","(3472694, 3672694)","(4162, 4382)","(4382, 4602)"
101,6321793,20221215_145401_comp_amd_om_6_1_and_6_3,20221215_145401_comp_amd_om_6_1_top_4_base_3,5272689,20221215_145401_comp_amd_om_6_1_and_6_3.1.vide...,6626,1.0,6_1_top_4_base_3,win_non_comp,20221215_145401_comp_amd_om_6_1_and_6_3.1,"(6.1, 6.3)",6.1,win,263634,"(253634, 263634)","(263634, 273634)","(5072689, 5272689)","(5272689, 5472689)","(6406, 6626)","(6626, 6846)"
102,7921790,20221215_145401_comp_amd_om_6_1_and_6_3,20221215_145401_comp_amd_om_6_1_top_4_base_3,6872686,20221215_145401_comp_amd_om_6_1_and_6_3.1.vide...,8620,1.0,6_1_top_4_base_3,win_non_comp,20221215_145401_comp_amd_om_6_1_and_6_3.1,"(6.1, 6.3)",6.1,win,343634,"(333634, 343634)","(343634, 353634)","(6672686, 6872686)","(6872686, 7072686)","(8400, 8620)","(8620, 8840)"
103,9421790,20221215_145401_comp_amd_om_6_1_and_6_3,20221215_145401_comp_amd_om_6_1_top_4_base_3,8372686,20221215_145401_comp_amd_om_6_1_and_6_3.1.vide...,10489,1.0,6_1_top_4_base_3,win_non_comp,20221215_145401_comp_amd_om_6_1_and_6_3.1,"(6.1, 6.3)",6.1,win,418634,"(408634, 418634)","(418634, 428634)","(8172686, 8372686)","(8372686, 8572686)","(10269, 10489)","(10489, 10709)"
104,11421788,20221215_145401_comp_amd_om_6_1_and_6_3,20221215_145401_comp_amd_om_6_1_top_4_base_3,10372684,20221215_145401_comp_amd_om_6_1_and_6_3.1.vide...,12983,1.0,6_1_top_4_base_3,win_non_comp,20221215_145401_comp_amd_om_6_1_and_6_3.1,"(6.1, 6.3)",6.1,win,518634,"(508634, 518634)","(518634, 528634)","(10172684, 10372684)","(10372684, 10572684)","(12763, 12983)","(12983, 13203)"
105,13321785,20221215_145401_comp_amd_om_6_1_and_6_3,20221215_145401_comp_amd_om_6_1_top_4_base_3,12272681,20221215_145401_comp_amd_om_6_1_and_6_3.1.vide...,15351,1.0,6_1_top_4_base_3,win_non_comp,20221215_145401_comp_amd_om_6_1_and_6_3.1,"(6.1, 6.3)",6.1,win,613634,"(603634, 613634)","(613634, 623634)","(12072681, 12272681)","(12272681, 12472681)","(15131, 15351)","(15351, 15571)"
106,15121782,20221215_145401_comp_amd_om_6_1_and_6_3,20221215_145401_comp_amd_om_6_1_top_4_base_3,14072678,20221215_145401_comp_amd_om_6_1_and_6_3.1.vide...,17595,1.0,6_1_top_4_base_3,win_non_comp,20221215_145401_comp_amd_om_6_1_and_6_3.1,"(6.1, 6.3)",6.1,win,703633,"(693633, 703633)","(703633, 713633)","(13872678, 14072678)","(14072678, 14272678)","(17375, 17595)","(17595, 17815)"
107,16721780,20221215_145401_comp_amd_om_6_1_and_6_3,20221215_145401_comp_amd_om_6_1_top_4_base_3,15672676,20221215_145401_comp_amd_om_6_1_and_6_3.1.vide...,19589,1.0,6_1_top_4_base_3,win_non_comp,20221215_145401_comp_amd_om_6_1_and_6_3.1,"(6.1, 6.3)",6.1,win,783633,"(773633, 783633)","(783633, 793633)","(15472676, 15672676)","(15672676, 15872676)","(19369, 19589)","(19589, 19809)"
108,18521777,20221215_145401_comp_amd_om_6_1_and_6_3,20221215_145401_comp_amd_om_6_1_top_4_base_3,17472673,20221215_145401_comp_amd_om_6_1_and_6_3.1.vide...,21833,1.0,6_1_top_4_base_3,win_non_comp,20221215_145401_comp_amd_om_6_1_and_6_3.1,"

In [65]:
current_recording.get_traces(channel_ids=["0"], start_frame=65692532, end_frame=67692532).T[0].shape

(1000000,)

In [51]:
79696960//20000/60

66.4

In [55]:
current_recording

SpikeGadgetsRecordingExtractor: 32 channels - 20.0kHz - 1 segments - 66,692,532 samples 
                                3,334.63s (55.58 minutes) - int16 dtype - 3.98 GiB
  file_path: /scratch/back_up/reward_competition_extention/data/omission/2022_12_15/20221215_145401_comp_amd_om_6_1_and_6_3.rec/20221215_145401_comp_amd_om_6_1_top_4_base_3.rec

In [ ]:
71,681,918

In [ ]:
current_recording

- Filtering for all trials that we got the LFP for

In [ ]:
TONE_TIMESTAMP_DF = TONE_TIMESTAMP_DF[TONE_TIMESTAMP_DF["recording_file"].isin(recording_name_to_all_ch_lfp.keys())].reset_index(drop=True)

In [ ]:
TONE_TIMESTAMP_DF.head()

- Adding trial numbers based on timestamp ordering for each recording

In [ ]:
TONE_TIMESTAMP_DF = TONE_TIMESTAMP_DF.groupby('recording_file').apply(lambda g: compute_sorted_index(g, value_column='time', index_column='trial_number')).reset_index(drop=True)

In [ ]:
TONE_TIMESTAMP_DF["trial_number"] = TONE_TIMESTAMP_DF["trial_number"] + 1

## Adding the LFP trace information

In [ ]:
CHANNEL_MAPPING_DF

- Adding all the brain region to ch information

In [ ]:
CHANNEL_MAPPING_DF["Subject"] = CHANNEL_MAPPING_DF["Subject"].astype(str)

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF = TONE_TIMESTAMP_DF.merge(CHANNEL_MAPPING_DF, left_on="current_subject", right_on="Subject", how="left")

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF = channel_map_and_TONE_TIMESTAMP_DF.drop(columns=[col for col in channel_map_and_TONE_TIMESTAMP_DF.columns if "eib" in col], errors="ignore")

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF = channel_map_and_TONE_TIMESTAMP_DF.drop(columns=["Subject"], errors="ignore")

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF.head()

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF.to_csv("./proc/trial_metadata.csv")

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF.to_pickle("./proc/trial_metadata.pkl")

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF.columns

- Linking up all LFP calculations with all the trials

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF["all_ch_lfp"] = channel_map_and_TONE_TIMESTAMP_DF["recording_file"].map(recording_name_to_all_ch_lfp)

- Creating a new row for each brain region

In [ ]:
brain_region_col = [col for col in CHANNEL_MAPPING_DF if "spike_interface" in col]

In [ ]:
id_cols = [col for col in channel_map_and_TONE_TIMESTAMP_DF.columns if col not in brain_region_col]

In [ ]:
brain_region_col

In [ ]:
for col in brain_region_col:
    channel_map_and_TONE_TIMESTAMP_DF[col] = channel_map_and_TONE_TIMESTAMP_DF[col].astype(int).astype(str)

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF.columns

- Extracting the traces for each trial and brain region

In [ ]:
for col in brain_region_col:
    print(col)
    brain_region = col.strip("spike_interface").strip("_")
    baseline_trace_column = "{}_baseline_lfp_trace".format(brain_region)
    trial_trace_column = "{}_trial_lfp_trace".format(brain_region)
    baselineandtrial_trace_column = "{}_baseline-trial_lfp_trace".format(brain_region)
    
    channel_map_and_TONE_TIMESTAMP_DF[baseline_trace_column] = channel_map_and_TONE_TIMESTAMP_DF.apply(lambda row: row["all_ch_lfp"].get_traces(channel_ids=[row[col]], start_frame=row["baseline_lfp_timestamp_range"][0], end_frame=row["baseline_lfp_timestamp_range"][1]).T[0], axis=1)

    channel_map_and_TONE_TIMESTAMP_DF[trial_trace_column] = channel_map_and_TONE_TIMESTAMP_DF.apply(lambda row: row["all_ch_lfp"].get_traces(channel_ids=[row[col]], start_frame=row["trial_lfp_timestamp_range"][0], end_frame=row["trial_lfp_timestamp_range"][1]).T[0], axis=1)

    channel_map_and_TONE_TIMESTAMP_DF[baselineandtrial_trace_column] = channel_map_and_TONE_TIMESTAMP_DF.apply(lambda row: np.concatenate([row[baseline_trace_column], row[trial_trace_column]]), axis=1)                                                                                                              
                                                                                                                                                           
                                                                                                                                                    

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF = channel_map_and_TONE_TIMESTAMP_DF.drop(columns=["all_ch_lfp"], errors="ignore")

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF = channel_map_and_TONE_TIMESTAMP_DF.drop(columns=[col for col in channel_map_and_TONE_TIMESTAMP_DF if "spike_interface" in col], errors="ignore")

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF.head()

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF.columns

In [ ]:
channel_map_and_TONE_TIMESTAMP_DF.to_pickle("./proc/full_baseline_and_trial_lfp_traces.pkl")